In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-191147
Azure region: southcentralus
Subscription id: 976ee174-3882-4721-b90a-b5fef6b72f24
Resource group: aml-quickstarts-191147


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "cpu-cluster"
cluster_name = "Manual-Compute"

# HK-Step 3: Create computer Cluster 
# I used the code from the nyc-taxi code example 
# 
# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name) 
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)


Found existing cluster, use it.

Running


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# HK-Step 4: Specify parameter sampler
# See the Keras example code
#
ps = RandomParameterSampling(
{
    '--C' : choice(0.01,0.1,1,10,100),
    '--max_iter' : choice(20,40,70,100)
})


# HK-Step 5:  Specify a Policy
# See the Keras example code
#
policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# HK-Step 6: Create a ScriptRunConfig Object to specify the configuration details of your training job
# See the Keras example code
#

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=aml_compute,
                      environment=sklearn_env)

# HK-Step 7:Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
# See the Keras example code
#
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)


In [9]:
# HK-Step 7: Submit your hyperdrive run to the experiment and show run details with the widget.
# See the Keras example code
#
hyperdrive_run = exp.submit(config=hyperdrive_config)

# We can use a run history widget to show the progress
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

#hyperdrive_run.get_status()

assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_79914e2f-00ae-4b36-884e-019c606195b5
Web View: https://ml.azure.com/runs/HD_79914e2f-00ae-4b36-884e-019c606195b5?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-191147/workspaces/quick-starts-ws-191147&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-04-06T08:22:10.244738][API][INFO]Experiment created<END>\n""<START>[2022-04-06T08:22:10.970772][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-04-06T08:22:11.583361][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_79914e2f-00ae-4b36-884e-019c606195b5
Web View: https://ml.azure.com/runs/HD_79914e2f-00ae-4b36-884e-019c606195b5?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-191147/workspaces/quick-starts-ws-191147&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



KeyError: 'log_files'

In [ ]:
import joblib
# HK-Step 8: Get your best run and save the model from that run.
# See the Keras example code
# 

# Keras code:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
model = best_run.register_model(model_name='keras-mlp-mnist', model_path='outputs/model')

# Number 5 solution #############
hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("best run details :", hyperdrive_best_run.get_details())
print('\n')
print("best run file names :", hyperdrive_best_run.get_file_names())
print('\n')
print("best run metrics :", hyperdrive_best_run.get_metrics())

model = hyperdrive_best_run.register_model(model_name='hyperdrive_log_reg_best_run', 
model_path='outputs/model.joblib', tags={"Method" : "HyperDrive"},
properties={"Accuracy" : hyperdrive_best_run.get_metrics()["Accuracy"]})

#Number 1 solution ####################

best_run = hyperdrive_submission.get_best_run_by_primary_metric()
print('run_id : ',best_run.id)
print('run_metrics : ',best_run.get_metrics())

final_model = best_run.register_model(model_name = 'hypermodel',model_path = '/outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# HK-Step 9: Create TabularDataset using TabularDatasetFactory
# 

data_location = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path=data_location)


In [ ]:
from train import clean_data

# HK-Step 10: Use the clean_data function to clean your data.
#
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)


In [2]:
# Submit your automl run

remote_run = exp.submit(config=automl_config, show_output=True)
remote_run.wait_for_completion()


In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

# Model 3
best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())

#Model 5
remote_run.get_metrics()

#Model 1
best_class_automl, best_fit_model = class_automl.get_output()
print(best_fit_model)

joblib.dump(best_fit_model, 'outputs/best_fit_automl_model.joblib')
best_class_automl.register_model(model_name = 'best_automl.pkl', model_path = '/outputs/')